<a href="https://colab.research.google.com/github/FabbriMatteo6/VideoTranscription/blob/main/Video_Transcription_with_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Setup and Install Libraries

This first cell will install the required Python libraries. It also runs a command to prove that ffmpeg is already available in the Colab environment.

In [ ]:
# Install the required libraries quietly
!pip install -q moviepy openai-whisper

# Verify that ffmpeg is installed and available
!ffmpeg -version

print("\n✅ Libraries installed and ffmpeg is available.")

Step 2: Upload Your Video File

Run this cell. It will display a "Choose Files" button. Click it and select the video file from your computer that you want to transcribe.

Wait for the upload to complete (it will show "100% done") before moving to the next step.

In [ ]:
from google.colab import files
import os

# Prompt the user to upload a file
uploaded = files.upload()

# Get the name of the uploaded file and print it
if uploaded:
    input_video_file = next(iter(uploaded))
    print(f"\n✅ Successfully uploaded file: '{input_video_file}'")
else:
    print("\n⚠️ No file was uploaded.")

Step 3: Define the Processing Functions

This cell contains the core Python functions that will handle the audio extraction and transcription. You just need to run this cell once to make the functions available for the next step.

In [ ]:
import whisper
from moviepy.video.io.VideoFileClip import VideoFileClip

def extract_audio(video_path):
    """Extracts audio from a video and saves it as an MP3."""
    audio_path = None
    try:
        print(f"Extracting audio from '{video_path}'...")
        video_clip = VideoFileClip(video_path)

        # Create a path for the output audio file
        base_name = os.path.splitext(video_path)[0]
        audio_path = f"{base_name}.mp3"

        # Write the audio file
        video_clip.audio.write_audiofile(audio_path, codec='mp3', logger=None)
        video_clip.close()
        print(f"✅ Audio successfully extracted to '{audio_path}'")
        return audio_path
    except Exception as e:
        print(f"❌ An error occurred during audio extraction: {e}")
        return None

def transcribe_audio(audio_path):
    """Transcribes an audio file using Whisper."""
    if not audio_path:
        return None

    try:
        print("Loading the Whisper 'large-v3-turbo' model...")
        model = whisper.load_model("large-v3-turbo")

        print(f"Transcribing '{audio_path}'... (this may take a while)")
        result = model.transcribe(audio_path)
        print("✅ Transcription complete.")
        return result["text"]
    except Exception as e:
        print(f"❌ An error occurred during transcription: {e}")
        return None

Step 4: Run the Full Process

This cell executes the entire workflow. It will call the functions defined above to extract the audio, transcribe it, and save the result to a text file. It will also clean up the temporary audio file.

In [ ]:
audio_file_path = None
text_file_path = None

try:
    if 'input_video_file' in locals() and input_video_file:
        # Step 1: Extract audio
        audio_file_path = extract_audio(input_video_file)

        # Step 2: Transcribe the audio
        transcribed_text = transcribe_audio(audio_file_path)

        if transcribed_text:
            # Step 3: Save the transcription to a text file
            base_name = os.path.splitext(input_video_file)[0]
            text_file_path = f"{base_name}.txt"
            with open(text_file_path, 'w', encoding='utf-8') as f:
                f.write(transcribed_text)
            print(f"\n✅ Success! Transcription saved to '{text_file_path}'")
        else:
            print("\n⚠️ Could not generate transcription.")
    else:
        print("\n⚠️ Please go back to Step 2 and upload a video file first.")

finally:
    # Step 4: Clean up the temporary audio file
    if audio_file_path and os.path.exists(audio_file_path):
        print(f"Cleaning up temporary file: '{audio_file_path}'...")
        os.remove(audio_file_path)

Step 5: Download the Transcription File

Run this final cell. Your browser will automatically start downloading the .txt file containing your video's transcription.

In [ ]:
if 'text_file_path' in locals() and text_file_path and os.path.exists(text_file_path):
    print(f"Downloading '{text_file_path}'...")
    files.download(text_file_path)
else:
    print("⚠️ No transcription file to download. Please ensure Step 4 completed successfully.")